In [142]:
#!/usr/bin/env python
# coding: utf-8
import tkinter
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib

def get_main_df():
    #reading the csv, setting the linebreak as 'tab', setting datetime format for the Month index
    #I am setting the index as Month so that we can apply formatting parameters, which will be conserved even as we reset the index a few times within the function
    #i.e. the index will be changed a few times in this function, but at the end it will be set as multi-index for Month and Region, and these parameters will still work then
    main_df = pd.read_csv(Path('data/redfin_table.csv'), on_bad_lines='skip', sep='\t', infer_datetime_format=True, parse_dates=True, index_col=['Month'])
   
    #add state code column using the last two letters from the Region column, and set it as the index. This will be used as a common column for merging the second DF.
    main_df['state_code'] = main_df['Region'].apply(lambda x: x[-2:])
    main_df.reset_index(inplace=True)
    main_df.set_index(keys=['state_code'], inplace=True)
    
    #create DF for tax rates from CSV and merge it to the main DF using the state_code column as the merge point
    #this adds tax rate column to the main DF
    tax_df = pd.read_csv("./data/tax_rates_state.csv")
    tax_df = tax_df.drop('state', axis=1)
    tax_df['tax_rate'] = tax_df['tax_rate'].str.replace(r'%', '', regex=True).astype('float')/100
    tax_df= tax_df.query("state_code == ('MA', 'TX', 'CA', 'GA', 'DC', 'FL', 'IN', 'PA', 'CO', 'MI')")
    tax_df = tax_df.set_index(keys='state_code')
    
    #merge the two DF's
    main_df = pd.merge(main_df, tax_df, on='state_code')
    
    #reset index again, this time as month and region; this is the final index format
    main_df.reset_index(inplace=True)
    main_df.set_index(keys=['Month', 'Region'], inplace=True)
    main_df.sort_index(level=['Month', 'Region'], ascending=[1, 0], inplace=True)
    
    #shorten the DF to only include the 10 chosen cities
    main_df = main_df.loc[slice(None), ['Austin, TX', 'San Diego, CA', 'Atlanta, GA', 'Washington, DC', 'West Palm Beach, FL', 'Indianapolis, IN', 'Pittsburgh, PA', 'Detroit, MI', 'Denver, CO', 'Boston, MA'], : ]
    
    #drop unnecessary columns
    main_df = main_df.drop(['For-Sale Property Type', 'state_code'], axis=1)
    
    #clean data of $/%/,
    cols = ['Average Monthly Rent', 'Average Rent YoY', 'Monthly mortgage, 5% down', 'Monthly mortgage, 5% down YoY', 'Monthly mortgage, 20% down', 'Monthly mortgage, 20% down YoY', 'Median sale price', 'Median sale price YoY'] 
    main_df[cols] = main_df[cols].replace({'\$': '', ',': '', '\%': ''}, regex=True)
    
    #set data as int/float where needed
    main_df['Average Monthly Rent'] = main_df['Average Monthly Rent'].astype('int')
    main_df['Monthly mortgage, 5% down'] = main_df['Monthly mortgage, 5% down'].astype('int')
    main_df['Monthly mortgage, 20% down'] = main_df['Monthly mortgage, 20% down'].astype('int')
    main_df['Median sale price']=main_df['Median sale price'].astype('int')
    main_df['Average Rent YoY']=main_df['Average Rent YoY'].astype('float')
    main_df['Monthly mortgage, 5% down YoY']=main_df['Monthly mortgage, 5% down YoY'].astype('float')
    main_df['Monthly mortgage, 20% down YoY']=main_df['Monthly mortgage, 20% down YoY'].astype('float')
    main_df['Median sale price YoY']=main_df['Median sale price YoY'].astype('float')
    
    return main_df

def get_city_list():
    df = get_main_df()
    df.reset_index(inplace=True)
    return df['Region'].unique().tolist()

#the main_df now has a column for tax rates, might not need this function
def get_tax_rates():
    # Return a data frame with columns Region, state_code, tax_rate
    #   for all cities in our main dataframe
    main_df = get_main_df()
    main_df.reset_index(inplace=True)
    df = pd.DataFrame(main_df.Region.unique(), columns=(['Region']))
    df['state_code'] = df['Region'].apply(lambda x: x[-2:])
    tax_df = pd.read_csv("./data/tax_rates_state.csv")
    df_final = pd.merge(df, tax_df, how='right', on=['state_code', 'state_code'])
    df_final.dropna(inplace=True)
    df_final['tax_rate'] = df_final['tax_rate'].str.replace(r'%', '', regex=True).astype('float') / 100
    df_final.reset_index(inplace=True, drop=True)
    return df_final


In [143]:
main_df = get_main_df()
main_df

Average Monthly Rent  Average Rent YoY  \
Month      Region                                               
2021-02-01 Austin, TX                  1644               NaN   
           Austin, TX                  1644               NaN   
           Austin, TX                  1644               NaN   
           Austin, TX                  1644               NaN   
           Austin, TX                  1644               NaN   
...                                     ...               ...   
2022-06-01 Boston, MA                  4031              17.2   
           Boston, MA                  4031              17.2   
           Boston, MA                  4031              17.2   
           Boston, MA                  4031              17.2   
           Boston, MA                  4031              17.2   

                       Monthly mortgage, 5% down  \
Month      Region                                  
2021-02-01 Austin, TX                       1552   
           Austin, TX                       1414   
           Austin, TX                       1810   
           Austin, TX                       1563   
           Austin, TX                       1126   
...                                          ...   
2022-06-01 Boston, MA                       3811   
           Boston, MA                       3297   
           Boston, MA                       4423   
           Boston, MA                       4000   
           Boston, MA                       3250   

                       Monthly mortgage, 5% down YoY  \
Month      Region                                      
2021-02-01 Austin, TX                            NaN   
           Austin, TX                            NaN   
           Austin, TX                            NaN   
           Austin, TX                            NaN   
           Austin, TX                            NaN   
...                                              ...   
2022-06-01 Boston, MA                           46.8   
           Boston, MA                           46.6   
           Boston, MA                           47.6   
           Boston, MA                           45.1   
           Boston, MA                           50.9   

                       Monthly mortgage, 20% down  \
Month      Region                                   
2021-02-01 Austin, TX                        1307   
           Austin, TX                        1191   
           Austin, TX                        1524   
           Austin, TX                        1317   
           Austin, TX                         948   
...                                           ...   
2022-06-01 Boston, MA                        3209   
           Boston, MA                        2776   
           Boston, MA                        3725   
           Boston, MA                        3369   
           Boston, MA                        2737   

                       Monthly mortgage, 20% down YoY  Median sale price  \
Month      Region                                                          
2021-02-01 Austin, TX                             NaN             397184   
           Austin, TX                             NaN             361750   
           Austin, TX                             NaN             463000   
           Austin, TX                             NaN             400000   
           Austin, TX                             NaN             288050   
...                                               ...                ...   
2022-06-01 Boston, MA                            46.8             705000   
           Boston, MA                            46.6             609900   
           Boston, MA                            47.6             818250   
           Boston, MA                            45.1             740000   
           Boston, MA                            50.9             601250   

                       Median sale price YoY  tax_rate  
Month      Region                  